# T5を使用した学習

## Google Driveとの連携

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ln -s "/content/drive/My Drive/ColabNotebooks" "/content/ColabNotebooks"

## 必要パッケージのインストール

In [ ]:
!pip install -r /content/ColabNotebooks/requirements.txt

## 学習

### パッケージのimport

In [ ]:
import pandas as pd
import tokenizers
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
    AdamW,
    get_linear_schedule_with_warmup,
)
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from evaluate import load as load_metric
import os
import math

### 転移学習モデルのロード

#### モデルパスの定義

In [ ]:
MODEL_SELECTIONS = (
    "t5-small",
    "t5-base",
    "t5-large",
    "google/flan-t5-small",
    "google/flan-t5-base",
    "google/flan-t5-large",
    "google/flan-t5-xl",
    "google/flan-t5-xxl",
    "retrieva-jp/t5-small-short",
    "retrieva-jp/t5-small-medium",
    "retrieva-jp/t5-small-long",
    "retrieva-jp/t5-base-short",
    "retrieva-jp/t5-base-medium",
    "retrieva-jp/t5-base-long",
    "retrieva-jp/t5-large-short",
    "retrieva-jp/t5-large-medium",
    "retrieva-jp/t5-large-long",
    "retrieva-jp/t5-xl",
    "sonoisa/t5-base-english-japanese",
    "sonoisa/t5-base-japanese",
    "sonoisa/t5-base-japanese-question-generation",
)
model_path = MODEL_SELECTIONS[4]

#### モデルの読み込み

In [ ]:
# トークナイザとモデルの準備
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

### 固定値の設定

In [ ]:
# 生成元のシーケンスの最大長を定義します。
max_length_src = 32

# 生成されるシーケンスの最大長を定義します。
# これは問題の性質によって異なりますが、
# 一般的にはソーステキストの長さの2倍程度を指定することが推奨されます。
max_length_target = 64

# エポックとは、全ての訓練データがネットワークを通過する回数を指します。
# この値はモデルが過学習しないようにするために調整する必要があります。
# 一般的には、数エポック（2〜10など）から始め、
# 過学習の兆候を見つけるために検証セットのパフォーマンスを監視します。
num_train_epochs = 3

# 各デバイス（GPUまたはCPU）での訓練バッチサイズです。
# バッチサイズとは、同時にネットワークを通過するトレーニングデータの数を指します。
# この値は、利用可能なメモリに応じて適切に調整する必要があります。
# 一般的には、8から32の範囲が適切な値とされています。
per_device_train_batch_size=16

# 各デバイス（GPUまたはCPU）での評価バッチサイズです。
# 訓練バッチサイズ同様、利用可能なメモリに応じて調整する必要があります。
# 一般的には、訓練バッチサイズと同等またはそれ以上の値が選択されます。
per_device_eval_batch_size=8

# ウォームアップステップの数は、学習率スケジューリングに影響します。
# 学習の初期段階で学習率を徐々に上げることで、訓練の安定性を向上させることができます。
# 適切な値は訓練ステップの総数の一部で、
# 一般的にはトータルステップの10%程度が指定されます。
warmup_steps=1000

# 重み減衰は正則化手法の一つで、モデルの複雑さを制限することで過学習を防ぎます。
# 一般的には0.0から0.01の範囲が推奨されます。
weight_decay=0.01

# 学習率はモデルのパラメータ更新のスピードを制御します。
# 大きすぎると学習が不安定になり、小さすぎると学習が遅くなります。
# 一般的には0.00001（1e-5）から0.1の範囲で設定します。
learning_rate=5e-5

# 勾配累積ステップ数は、バッチサイズを事実上増加させるために使用されます。
# これにより、ハードウェアのメモリ制限に直面している場合でも、大きなバッチで訓練を行うことができます。
# 適切な値はハードウェアのメモリとバッチサイズに依存します。
# 例えば、バッチサイズが8で、メモリがそれ以上のバッチサイズを扱うことができない場合、
# 勾配累積ステップ数を2に設定することで、実質的なバッチサイズを16にすることができます。
# ただし、勾配累積は計算時間を増加させる可能性があります。
gradient_accumulation_steps=1

prefix = "translate English to Japanese: "
# prefix = "translate English to French: "

### 関連クラス・メソッドの定義

#### データセット用のクラスの作成

In [ ]:
class WordTranslationDataset(Dataset):
    def __init__(self, tokenizer, df):
        self.tokenizer = tokenizer
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        english_word = prefix + row["English"]
        wrong_translation = row["Japanese"]
        encoding = self.tokenizer.encode_plus(
            english_word,
            max_length=max_length_src,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        targets = self.tokenizer.encode_plus(
            wrong_translation,
            max_length=max_length_target,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        labels = targets["input_ids"].flatten()
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": labels,
        }

#### オプチマイザ、スケジューラー作成用メソッド

In [ ]:
def get_optimizers(training_steps):
  optimizer = AdamW(model.parameters(), lr=learning_rate)
  # 線形に減衰するスケジューラー
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=warmup_steps,
      num_training_steps=training_steps,
  )
  return optimizer, scheduler

#### 評価用メソッド

In [ ]:
# BLEUメトリクスの読み込み
bleu_metric = load_metric("bleu")

# BLEUスコアでの検証を行う
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(
        predictions, skip_special_tokens=True
    )
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu_score = bleu_metric.compute(
        predictions=decoded_preds, references=decoded_labels
    )
    print(f"bleu_score: {bleu_score['score']}")

    return {"bleu_score": bleu_score["score"]}

### 学習の実行

In [ ]:
# データの読み込みと前処理
# 英単語と誤った日本語訳のペアのcsv
df = pd.read_csv('/content/ColabNotebooks/input/en_and_ja.csv')
train_df, eval_df = train_test_split(df, test_size=0.2)

# データセットの作成
train_dataset = WordTranslationDataset(tokenizer, train_df)
eval_dataset = WordTranslationDataset(tokenizer, eval_df)

### GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    model.cuda()

# 最大ステップ数を計算(最低１回)
max_steps = max(
    1,
    math.ceil(len(train_dataset) / per_device_train_batch_size)
    * num_train_epochs,
)

# トレーニングの設定
training_args = Seq2SeqTrainingArguments(
    output_dir='results',  
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,  
    per_device_eval_batch_size=per_device_eval_batch_size,
    weight_decay=weight_decay,
    gradient_accumulation_steps=gradient_accumulation_steps,
    logging_dir='logs',        
)

optimizers = get_optimizers(max_steps)

# トレーナーの作成
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# 訓練
trainer.train()

# 訓練が終了したモデルとトークナイザを保存
model.save_pretrained("/content/ColabNotebooks/model/")
tokenizer.save_pretrained('/content/ColabNotebooks/model/')